In [23]:
import time
print(f"Last updated on {time.asctime(time.gmtime())} UTC")

Last updated on Wed Dec 25 03:49:17 2019 UTC


### Purpose

This notebook would serve to show the design process of the Back-off n-grams Language Models (BNLM) with enhancement from Neural Network Language Models (NNLM) as described the paper "Neural Network Language Model for Chinese Pinyin Input Method Engine" by Chen et all. The task at hand with this language model is given a sequence of syllables, to predict which is most likely the next syllable, a task also known as candidate sentence generation. The model is to be implemented into HKIME, an intelligent input method for Cantonese. There would be three sections in this notebook, each building upon the previous. 

### Breakdown

- Section 1: Basic n-grams probabilistic particle filter
- Section 2: Back-off n-grams language model with interpolated Kneser-Ney smoothing
- Section 3: BNLM (from section 2) with probabilities calculated with NNLM


In [24]:
import random

# Jyutping Corpus Processing

In [25]:
#TODO: Add corpus processing
#TODO: Find better corpuses
#TODO: Look into webscraping to generate corpuses ourselves

FILENAME = "sources/lihkg_corpus"

with open(FILENAME, "r") as f:
    content = f.read()
    jyutping_corpus = content.splitlines()
print(jyutping_corpus)

['海港城於周二平安夜發生一連串警民衝突，最少兩名市民被捕，一名市民路經時被警方無理用警棍打頭而要送院。', '衝突發生過後，現場除了有市民和防暴警員外，佐敦南選區候任區議員陳梓維亦在場。', '他不單在場用手機進行直播，告知街坊海港城的最新情況，更表示希望他在場能 唔好畀佢 警方 亂咁濫用暴力 。', '', '', '陳梓維原本受邀在平安夜晚上到一間教會參與活動，但當他得知海港城內有衝突發生，便決定中途離開，趕到海港城現場；到場時，港威商場三樓已全層封閉，二樓亦有防暴警員戒備。', '同時，他亦見到該名較早前頭部受傷的市民在現場接受救護員治療。', '進入海港城前，陳梓維已在專頁 開 ；到場後，他眼見現場環境頗為混亂，遂決定繼續直播，向街坊進行現場報道。', '', '', '訪問途中，陳梓維提到一個小插曲，指他到場後，有一名防暴警員認得他為候任區議員，要求他 控制、處理現場環境 。', '陳梓維表示，警員當時的一番說話 即時引起全場哄動 。', '最後，該名警員在現場市民的叫嚷聲中，退至商場出入口處。', '作為候任區議員，陳梓維認為自己在場，可以 唔好畀佢 警方 亂咁濫用暴力 ，並且表示即使警方要進行拘捕，但都一定不可以使用過份武力。', '', '', '最後，記者問陳梓維在平安夜有甚麼說話想對香港人說？', '陳梓維不禁一笑，並表示希望每位香港人 平安夜能夠安全回家 ，並且能開心快樂地度過這個節日。', '', '', '當訪問完結後，又有警員走前並手指陳梓維說，這就是 會考零分都做到區議員嗰個人 。', '諷刺的是，那位警員在揶揄陳梓維的會考分數，卻不知道自己錯把陳梓維名字中的 梓  正確讀音 趾 誤讀為 辛 ，令在進行手機直播中的陳梓維，都忍不住當場糾正，令對方尷尬非常。 ']


# Section 1: basic n-grams prediction model

Post-processing of the cantonese corpus would get us a list of strings, where each string could be a phrase, a sentence, or a paragraph. For conciseness, we would call all of these sentences. In this section, we would divide up each sentence into the n-grams and then store the possible next letters for each n-gram in a python dictionary. The naive prediction algorithm would randomly pick from the possible next letters given a certain n-gram to generate candidate sentences.

### Set n-grams character count (the n in n-grams)

In [26]:
CHARACTER_COUNT = 2

### Generating n-grams dictionary

This would generate a dictionary where each key is an n-gram and the value would be a list of possible next characters.

In [27]:
#returns dictionary for prediction
def generate_n_grams_dict(processed_corpus):
    result = dict()
    for sentence in processed_corpus:
        #i is the start index of the slice
        for i in range(len(sentence) - CHARACTER_COUNT - 1): #-1 since last slice does not have next char
            grams = sentence[i:i+CHARACTER_COUNT]
            next_char = sentence[i+CHARACTER_COUNT]
            if grams in result:
                result[grams].append(next_char)
            else:
                #as long as there is an n-gram key in the dict, there would be at least one next char
                result[grams] = [next_char] 
    return result

### TODO: visualization of n-grams dict

### Prediction Model

This naive prediction model would, if the sentence has an n-gram in the dictionary, randomly select a next character from the list of potential next characters.

In [28]:
#returns a next character given an n_grams_dict and a sentence.
def predict_next_char(n_grams_dict, sentence):
    potentials = n_grams_dict.get(sentence[-CHARACTER_COUNT:], None)
    return random.choice(potentials) if potentials != None else None

### Testing

Here we would test the implementation of the naive n-grams prediction model, for comparison with more sophisticated language models. We would do two tests, the first one would generate a 200 character sentence, and the second would test the implementation analytically by seeing how many next characters it will predict correctly on the test dataset.

TODO: Add a validation dataset. The current corpus is too small to be used both for training and validation.

#### Test 1

In [30]:
def testing(sentence):
    n_grams_dict = generate_n_grams_dict(jyutping_corpus)
    tmp = sentence
    #Generate a sentence of up to 200 characters, will break if an n-gram not found in n-grams dict.
    for i in range(200):
        res = predict_next_char(n_grams_dict, tmp)
        if res == None:
            break
        else:
            tmp = tmp + res
    return tmp

# Observe that due to the stochastic nature of
# particle filtering, the result is not
# pre-determinable
print("Trial 1: ")
print(testing("最後"))
print("Trial 2: ")
print(testing("最後"))

Trial 1: 
最後，該名警員走前並手指陳梓維不禁一笑，並且表示希望他在場用手機進行拘捕，但當他得知海港城內有衝突發生過後，記者問陳梓維已在專頁 開 ；到場時，港威商場出入口處
Trial 2: 
最後，他眼見現場除了有市民和防暴警員當時的一番說話想對香港人 平安夜有甚麼說話 即時引起全場哄動 


#### Test 2
**WARNING** Not a validation dataset

In [31]:
n_grams_dict = generate_n_grams_dict(jyutping_corpus)
count = 0
correct = 0
for sentence in jyutping_corpus:
    for i in range(len(sentence) - CHARACTER_COUNT - 1):
        if predict_next_char(n_grams_dict, sentence[:i+CHARACTER_COUNT]) == sentence[i+CHARACTER_COUNT]:
            correct += 1
        count += 1

print(f"Total of {count} predictions made")
print(f"{correct} predictions correct")
print(f"Prediction accuracy: {correct/(count+1)}%") #TODO: Remove the +1 after there is content in the jyutping_corpus

Total of 629 predictions made
552 predictions correct
Prediction accuracy: 0.8761904761904762%


## Section 2: Back-off n-grams language model with interpolated Kneser-Ney smoothing